# CREASE for Vesicles

## Introduction

**This tutorial has been created by Ziyu Ye and Arthi Jayaraman to accompany: Ye, Z.; Jayaraman, A. Computational Reverse-Engineering Analysis for Scattering Experiments (CREASE) on Vesicles Assembled from Amphiphilic Macromolecular Solutions**

Computational Reverse Engineering Analysis for Scattering Experiments (CREASE) is a two step approach that combines a genetic algorithm (GA) as the first step with molecular simulations based reconstruction as the second step to determine structural features of assembled structures from aggregate dimensions to the chain- and monomer- level packing for a given input scattering profile. 

If using this code, please cite the following references for the method:

[1] Beltran-Villegas, D. J.; Wessels, M. G.; Lee, J. Y.; Song, Y.; Wooley, K. L.; Pochan, D. J.; Jayaraman, A. Computational Reverse-Engineering Analysis for Scattering Experiments on Amphiphilic Block Polymer Solutions. *J. Am. Chem. Soc.* **2019**, 141, 14916−14930.

[2] Wessels, M. G.; Jayaraman, A. Computational Reverse-Engineering Analysis of Scattering Experiments (CREASE) on Amphiphilic Block Polymer Solutions: Cylindrical and Fibrillar Assembly. *Macromolecules* **2021**, 54, 783-796.

[3] Wessels, M. G.; Jayaraman, A. Machine Learning Enhanced Computational Reverse Engineering Analysis for Scattering Experiments (CREASE) to Determine Structures in Amphiphilic Polymer Solutions. *ACS Polym. Au* **2021**, https://doi.org/10.1021/acspolymersau.1c00015.

[4] Heil, C.; Jayaraman, A. Computational reverse-engineering analysis for scattering experiments of assembled binary mixture of nanoparticles. *ACS Mater. Au* **2021**, https://doi.org/10.1021/acsmaterialsau.1c00015

[5] Ye, Z.; Jayaraman, A. Computational Reverse-Engineering Analysis for Scattering Experiments (CREASE) on Vesicles Assembled from Amphiphilic Macromolecular Solutions.

This notebook presents a tutorial on how the first step of CREASE, the GA, works in determining the structure of a vesicle assembly from small angle scattering intensity profiles. The GA code presented here is applicable to vesicle assemblies enclosed by one bilayer and can be adapted to other structure geometries by modification to sections of the code that will be highlighted as such.

The GA code for vesicles is able to determine:
1. Vesicle dimensions in core radius ($R_{core}$), the three individual layer thicknesses in the bilayer that consist of the inner solvophilic A layer ($t_{Ain}$), middle solvophobic B layer ($t_{B}$), and the outer solvophilic A layer ($t_{Aput}$).
2. How the solvophilic scatterers are split between inner ($s_{Ain}$) and outer ($1-s_{Ain}$) layers.
3. Polydispersity in vesicle size as implemented in the core radius dimension ($\sigma_{R}$) during I(q) calculation.

<img src="CREASE_vesicles_code_structure_dimensions.png" width=300 height=300 />

Here is the general structure showing the flow of the code:

<img src="code_diagram1.png" width=600 height=600 />

## Setup python libraries

In [1]:
import crease_ga as cga
from inspect import getsource

### Model.py

Current "location" in the code flow diagram marked in red:

<img src="code_diagram2.png" width=600 height=600 />

Let's start with the initial arguments that the user needs to choose and input for their system. 

In the cell block below, the user needs to define two sets of arguments: those that defines the basic architecture of the genetic algorithm, includng:
* number of individuals in a population (pop_number)
* number of generations that GA runs for (generations)
* number of binary digits used to represent each individual (nloci)

The user chooses the number of individuals in the population and the number of generations to run the GA (note: in this example we use 5 for both the number of individuals in the population and the number of generations to keep the calculations computationally feasible in this tutorial. To produce accurate matches to the target structure's $I$(q) one would need to use significantly higher number of individuals and generations on more powerful computer.). For each of the parameters to be fit (or genome) using GA, it is encoded as a seven-digit (chosen by the user as $n_{loci}$, here set as 7) binary sequence that is evenly distributed between the maximum and minimum bounds set by the user.

The second set of arguments are passed in with the `load_chemistry` method and define the shape of interest. In this example, we load in the shape `vesicle`, and set up some shape descriptors (`chemistry_params`) that a user needs to define. The 7 shape descriptors here are:
* number of scatterers per chain ($num_{scatterers}$)
* Number of beads on chain ($N$)
* density/volume fraction of beads in B layer ($rho_B$)
* Angstrom 'monomer contour length' ($l_{mono,b}）
* Angstrom 'monomer cminontour length' ($l_{mono,a}）
* fraction of B type monomers in chain ($fb$)
* number of replicates ($n_{LP}$)

The A and B type scatterers are randomly placed within the designated vesicle layer boundaries (more details to be given later) and the ratio of A and B scatterers matches the composition of the amphiphilic macromolecule ( 1−𝑓𝑏 ,  𝑓𝑏 ). These A and B scatterers could be point scatterers or scatterers with a specified diameter, as is the case in this example. Regardless of the choice, the total number of A and B scatterers is another choice the user has to make to balance the accuracy of the output vesicle dimensions from GA and the computational intensity of the  𝐼𝑐𝑜𝑚𝑝 (q) calculation (more details to be given later).

NOTE: The determination of  𝐼𝑐𝑜𝑚𝑝 (q) in the GA only uses the scatterer placements.

In this vesicle system example, the user can enter the molecular information about the solution that was used to obtain the scattering profile to guide the choice of number of scatterers placed in the individual candidate structure. Here for this tutorial example, we use the molecular information for a diblock copolymer with the solvophilic block A and solvophobic block B forming the vesicle bilayer. The diblock chain has total number of  𝑁  beads with  𝑓𝑏  fraction of the beads in the chain being solvophobic. The packing density of the solvophobic beads in the middle layer is given by  𝜌𝐵  and the diameters of the A an B beads (or "monomer contour length") are given by  𝑙𝑚𝑜𝑛𝑜,𝑎  and  𝑙𝑚𝑜𝑛𝑜,𝑏 , respectively. The specific values used in this example are given below.

For this vesicle system example, we calculate each individual's  𝐼𝑐𝑜𝑚𝑝 (q) using seven different configurations (nLP) of random scatterer placements to generate the averaged  𝐼𝑐𝑜𝑚𝑝 (q). Having multiple configurations of scatterer placements mitigates the chance of any one configuration of scatterer placements within the vesicle dimension from biasing the computed scattering intensity. One could also choose to replace the use of multiple (nLP=7in this case) placements of fewer scatterers with just one placement using a significantly higher total number of scatterers per unit volume (i.e., scatterer density) which will lead to reduced biasing one may see upon using fewer scatterers (or smaller scatterer density). We choose to use multiple different configurations of random scatterer placements in this case to incorporate the ability to capture dispersity (more details to be given later).

For a more detailed discussion on the scatterer placement and density in the case of CREASE for vesicles, please see reference [5].

Here is also where the user can set the maximum and minimum bounds for each parameter (or genome) of the individual. For the vesicle system example, the parameters that the GA determines are:
* core radius ($R_{core}$), 
* three individual layer thicknesses in the bilayer consisting of the inner solvophilic A layer ($t_{Ain}$), middle solvophobic B layer ($t_{B}$), and the outer solvophilic A layer ($t_{Aput}$), 
* split of the solvophilic scatterers between inner ($s_{Ain}$) and outer layers,
* polydispersity in vesicle size as implemented in the core radius dimension ($\sigma_{R}$).
* background intensity ($background$).

Different shapes will have different sets of shape descriptors and parameters. It is important for the user to make sure the shape descriptors and minimum/maximum bound defined matches the expectation of the corresponding shape.
**TODO: Add a reference to the table of shape descriptors and parameters**

In [11]:
m = cga.Model(pop_number = 5,
              generations = 5,
              nloci = 7)

m.load_chemistry(chemistry='vesicle',chemistry_params=[24,54,0.5,50.4,50.4,0.55,7],
                                     minvalu = (50, 30, 30, 30, 0.1, 0.0, 0.1),
                                     maxvalu = (400, 200, 200, 200, 0.45, 0.45, 4))

As the GA selects the individuals that gets to move on to the next generation, the selected individual can be modified through the crossover and mutation procedures. The selected individual undergoes crossover with probability $PC$ and mutation with probability $PM$, and the initial values of these two parameters are chosen below. The user also sets the upper and lower bounds for these two parameters ($PC_{max}$, $PC_{min}$, $PM_{max}$, $PM_{min}$). During the GA, $PC$ and $PM$ are updated based on the reciprocal genetic diversity parameter, $GDM$, given by

$$ GDM = \frac{\text{min error of population}}{\text{average error of population}} $$

which also has upper and lower bounds ($GDM_{max}$, $GDM_{min}$) set by the user. $GDM$ should be between 0 and 1. The form of the error will be described in more details in a later section. The adjustment factor $kGDM$ is also set by the user that is used to adjust the genetic diversity of the population.

For this tutorial, we are just going to use the default GDM adaptation parameters for the sake of simplicity.
Run the cell block below to see the parameters being used:

In [12]:
for d in dir(m.adaptation_params):
    if not '__' in d:
    
        print(d,getattr(m.adaptation_params,d))

gdmmax 0.85
gdmmin 0.005
kgdm 1.1
pc 0.6
pcmax 1
pcmin 0.5
pm 0.001
pmmax 0.25
pmmin 0.006
update <bound method adaptation_params.update of <crease_ga.adaptation_params.adaptation_params object at 0x7f91f8e4d280>>


**TODO: Teach the user how to set their own adaptation parameters.**

Taking in the above input values, the GA calls the **Initial_pop** function to create the initial population of individuals. So let's first take a look at the `Initial_pop` function.

### Initial_pop.py

<img src="code_diagram3.png" width=600 height=600 />

Takes in the population number and generate individuals based on the bounds set for each of the GA parameters, then output the population information ($pop$) back to the **GA_main** code.


To see the source code of this funtion, run following block:

In [13]:
print(getsource(cga.utils.initial_pop))

def initial_pop(popnumber, nloci, numvars):
    '''
    Generate a generation of (binary) chromosomes.
    
    Parameters
    ----------
    popnumber: int
        Number of individuals in a population.
    nloci: int
        Number of binary bits to represent each parameter in a chromosome.
    numvars: int
        Number of parameters in a chromosome.
        
    Return
    ------
    pop: np.array of size (`popnumber`,`nloci`*`numvars`)
        A numpy array of binary bits representing the entire generation, 
        with each row representing a chromosome.
    '''
    pop=np.zeros((popnumber,nloci*numvars))
    for i in range(popnumber):
        for j in range(nloci*numvars):
            randbinary=np.random.randint(2)
            pop[i,j]=randbinary
    return pop



Run the block below to see what a binary representation for an example of 5 individuals in the population looks like:

In [14]:
pop = cga.utils.initial_pop(popnumber = 5, nloci = 7, numvars = 7)
print(pop)

[[0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0.
  1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
  0.]
 [0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0.
  1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1.
  1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1.
  1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0.
  1.]
 [0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0.
  1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0.
  1.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1.
  1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
  1.]]


To read the docstring of this funtion, run following block:

In [15]:
print(cga.utils.initial_pop.__doc__)


    Generate a generation of (binary) chromosomes.
    
    Parameters
    ----------
    popnumber: int
        Number of individuals in a population.
    nloci: int
        Number of binary bits to represent each parameter in a chromosome.
    numvars: int
        Number of parameters in a chromosome.
        
    Return
    ------
    pop: np.array of size (`popnumber`,`nloci`*`numvars`)
        A numpy array of binary bits representing the entire generation, 
        with each row representing a chromosome.
    


### Model.py

<img src="code_diagram2.png" width=600 height=600 />

Back to the main GA code after creating the initial population of individuals. Now let's read in the "experimental" scattering intensity profile $I_{exp}(q)$ (`loadvals`) that in this example is stored in a text file that has the $q$ and $I(q)$ information. 

For the example here, our $I_{exp}(q)$ is an *in silico* $I(q)$ created from a target vesicle structure where all dimensions and dispersity in size are known and thus the GA can be tested on it to see how well the code performs in getting each parameter.

After loading in the target $I_{exp}(q)$, the user can set the upper and lower bounds on the $q$-range over which the GA will determine the best candidate vesicle structure. 

Run the cell block below to see how loading the input $I_{exp}(q)$ is implemented in the code.

In [16]:
print(getsource(cga.Model.load_iq))

    def load_iq(self,input_file_path,q_bounds=None):
        loadvals = np.loadtxt(input_file_path)
        self.qrange_load = loadvals[:,0]
        IQin_load = loadvals[:,1]
        self.IQin_load=np.true_divide(IQin_load,np.max(IQin_load))
        #TODO: highQ and lowQ needs to be able to be dynamically set
        if q_bounds == None:
            self.qrange = self.qrange_load
            self.IQin = self.IQin_load
        else:
            lowQ = q_bounds[0]
            highQ = q_bounds[1]
            self.IQin = self.IQin_load[ np.where(self.qrange_load>=lowQ)[0][0]:np.where(self.qrange_load>=highQ)[0][0] ]
            self.qrange = self.qrange_load[ np.where(self.qrange_load>=lowQ)[0][0]:np.where(self.qrange_load>=highQ)[0][0] ]
            

        baseline = self.IQin[0]
        self.IQin = np.true_divide(self.IQin,baseline)
        self.IQin_load = np.true_divide(self.IQin_load,baseline)



Now we can start running the GA. Starting with the inital population of individuals (0-th generation), we first need to determine the "fitness" of each individual, that is, how well does its computed $I_{comp}$(q) matches the target "experimental" $I_{exp}$(q). To facilitate the fitness calculation in the GA code, we call the `fitness` function from **fitness.py** from the main GA code. The fitness calculation requires several input arguments passed from the main GA code that is in binary form and it "decodes" to get the values of all of the GA parameters for each of the individuals in the population. 

So before running the `fitness` block, first we need to run **decode.py** for the `decode` function.

### decode.py

<img src="code_diagram5.png" width=600 height=600 />

Run the cell block below to see the source code of this function.

In [17]:
print(getsource(cga.utils.decode))

def decode(pop, indiv, nloci, minvalu,maxvalu):
    import numpy as np
    minvalu = np.array(minvalu)
    maxvalu = np.array(maxvalu)
    deltavalu=maxvalu-minvalu
    nvars=len(minvalu)
    valdec=np.zeros(nvars)
    param=np.zeros(nvars)
    #   decodes from binary to values between max and min
    for j in range(nvars): 
        n=nloci
        for i in range(j*nloci,(j+1)*nloci):
            n=n-1
            valdec[j]+=pop[indiv,i]*(2**n)
            
        param[j]=minvalu[j]+np.true_divide((deltavalu[j])*(valdec[j]),2**nloci)
    return param



To see an example of the decoded parameter values for an individual generated using `initial_pop`, run the following cell block:

In [18]:
pop = cga.utils.initial_pop(popnumber = 5, nloci = 7, numvars = 7)
cga.utils.decode(pop,0,7,[50, 30, 30, 30, 0.1, 0.0, 0.1],[400, 200, 200, 200, 0.45, 0.45, 4])

array([8.28125000e+01, 6.71875000e+01, 4.46093750e+01, 1.56171875e+02,
       3.07812500e-01, 2.10937500e-02, 1.59296875e+00])

Now let's take a closer look at what happens in **fitness.py** and how fitness is calculated for each individual in the population.

### fitness.py

<img src="code_diagram4.png" width=600 height=600 />

The core of the `fitness` function is given by the following set of equations

$$ fitness = X(sse_{max}-see)+Y $$

$$ sse = \sum w\left[log\left(\frac{I_{exp}(q)}{I_{comp}(q)}\right)\right]^2 $$

$$ X = (cs-1)\frac{max(sse_{max}-sse)}{max(sse_{max}-sse)-average(sse_{max}-sse)} $$

$$ Y = (1-X)average(sse_{max}-sse) $$

where $I_{exp}$(q) is the input target scattering intensity profile read in from the main GA code, $I_{comp}$(q) is the computed scattering intensity profile for the individual (to be discussed in more details later on), $sse$ is the sum of squared errors between $I_{comp}$(q) and $I_{exp}$(q) and $X$ and $Y$ serve to prevent low fitness solutions from being prematurely eliminated in the population.

In this example, we choose the form of the error function, `sse`, as the sum of the log difference of squared errors where the weighting factor, $w$, accounts for the log-scale spacing of the q values. For more details on the choice of this form for the error function for the systems explored with CREASE, see references [1-5].

We also note that user may choose to use a different form of sse as long as the choice of error function ensures appropriate error weighting in the range of q explored. For studies where the target $I_{exp}$(q) comes from experimental scattering with an associated uncertainty at each q, a commonly used formulation is $\chi^{2}$ to account for the errors during fitting optimization. For example, in SASVIEW uses $\chi^{2}$ to quantify the goodness of fit by taking the direct difference squared between the computed and target dataset over the fitted q range weighted by the squared error at each q. If the error associated with each q is known, $\chi^{2}$ can similarly be implemented into the fitness function in CREASE to account for uncertainty in I(q) at each q.

SASVIEW implementation of $\chi_{2}$: https://www.sasview.org/docs/user/qtgui/Perspectives/Fitting/residuals_help.html

In order to perform the fitness calculation, the key piece of information needed is the computed scattering intensity, $I_{comp}$(q), which **fitness.py** gets from the `converttoIQ` function. So let's now take a look at how CREASE determines the the computed scattering intensity, $I_{comp}$(q).

### converttoIQ.py

<img src="code_diagram6.png" width=600 height=600 />

**Equations to get computed scattering intensity**

The `converttoIQ` function calculates the computed scattering intensity $I_{comp}$(q) from the intra-vesicle structure factor $\omega$(q) (see LPOmega.py) using the following set of equations from scatterers placed within the boundaries of the candidate individual structure:

$$ I_{comp}(q) = F_{M}^2(q)S_{MM}(q) + I_{background} $$

$F^2_{M}(q)$ is the form factor given by
$$ F^2_{M}(q) = \sum_{\alpha\in{A,B}} \sum_{\beta\in{A,B}} b_{\alpha}b_{\beta}F_{GA, \alpha}(q)F_{GA, \beta}(q)\omega(q) $$

where $b_{A}$ and $b_{B}$ are the scattering length densities for A and B scatterers, $F_{GA, \alpha}(q)$ and $F_{GA, \beta}$ are the spherical form factors for A and B scatterers (see **LPFbead.py**), $\omega(q)$ is the intra-vesicle structure factor (see **LPOmega.py**) and $S_{MM}(q) = 1$ is the vesicle-vesicle structure factor for dilute solutions.


**Scatterer placements**

As mentioned in an above section, the essential information for the calculation of $I_{comp}$(q) is the scatterer placements and the user can directly choose the total number of scatterers ($n_{tot}$) to be placed in the assembly to control the scatterer density. For an assembly formed from molecules with two (or more) chemistries, the user also input the relative compositions for the chemical groups as $f_{a}$ and $f_{b}$ ($f_{c}$, etc.). The choice of the scatterer density appropriate to the assembly studied is chosen by the user. One way to obtain an estimate of $n_{tot}$, as shown in this tutorial example for vesicles, is through the coarse-grained molecular information of the molecule that make up the assembly.

In this vesicle system example, let us determine the number of scatterers for placing within each layer in the vesicle wall:

1. Get the core radius ($R_{core}$) as the averaged core radius ($R_{core, 0}$) plus the variation in core radius for each replicate ($\sigma_{R}$).
$$ R_{core} = R_{core, 0} + \sigma_{R} $$

2. Calculate the volume of the solvophobic B layer for the individual:
$$ V_{B} = \frac{4}{3}\pi \left[ (R_{core}+t_{Ain}+t_{B})^3 - (R_{core}+t_{Ain})^3 \right] $$

3. Get the number of chains/molecules in the structure:
$$ N_{chain} = \frac{\rho_{B}V_{B}}{Nf_{B}M_{B}} $$

4. The number of A ($n_{A}$) and B ($n_{B}$) scatterers are determined using the composition ($f_{A}$, $f_{B}$) of the respective block in the chain/molecule are given by
$$ n_{A} = N_{chain}n_{sct}f_{A} = N_{chain}n_{sct}(1-f_{B}) $$
$$ n_{B} = N_{chain}n_{sct}f_{B} $$
where $n_{sct}$ is the number of scatterers per chain/molecule selected by the user.

5. The number A scatterers the inner ($n_{Ain}$) and outer ($n_{Aout}$) solvophilic layers are given by
$$ n_{Ain} = n_{A}s_{Ain} $$
$$ n_{Aout} = n_{A}(1-s_{Ain}) $$
where $s_{Ain}$ is a parameter in the GA that gives the fraction of chains/molecules in the inner solvophilic layer. 

**The $s_{Ain}$ parameter, which is determined through the GA, is a feature of the vesicle assembly that cannot be readily determined through traditional analytical fittings of scattering profiles. The addition and determination of $s_{Ain}$ in CREASE for vesicles offers molecular insight to the distribution of solvophilic monomers between the inner and outer layers in the vesicle wall**.

Now with the set of information for the vesicle dimensions and how many scatterers go into each layer in the vesicle wall, we can generate the scatterer positions by calling the `genLP` function from **genLP.py**.

### genLP.py and gen_layer.py

<img src="code_diagram7.png" width=600 height=600 />

**NOTE: genLP.py is the part of the GA code that the user needs to modify to generate the scatterer positions for the assembly of interest.**

In this example, we get the coordinates for all scatterers in the vesicle with one bilayer. 

For the vesicle system example presented here, the `gen_layer` function defined in **gen_layer.py** places $n_{size}$ number of random point scatterers within a spherical shell defined by the inner ($r_{in}$) and outer ($r_{out}$) radii, where $n_{size}$ is the corresponding number of A or B type scatterers determined in **converttoIQ.py** for the given shell or layer.

**NOTE: `gen_layer` function is specific to this example system.**

**gen_LP.py** then calls the `gen_layer` function for each individual layers in the vesicle (in this example, the 2 solvophilic A and 1 solvophobic B layer that make up the bilayer) and gathers the scatterer coordinates for the complete vesicle to be passed back to **converttoIQ.py** for use in calculating the intra-vesicle structure factor $\omega(q)$.

In [5]:
print(getsource(cga.chemistries.vesicle.scatterer_generator.gen_layer))
print(getsource(cga.chemistries.vesicle.scatterer_generator.genLP))

def gen_layer(rin, rout, nsize):
        R = 1.0

        phi = np.random.uniform(0, 2*np.pi, size=(nsize))
        costheta = np.random.uniform(-1, 1, size=(nsize))
        u = np.random.uniform(rin**3, rout**3, size=(nsize))

        theta = np.arccos( costheta )
        r = R * np.cbrt( u )

        x = r * np.sin( theta ) * np.cos( phi )
        y = r * np.sin( theta ) * np.sin( phi )
        z = r * np.cos( theta )

        return( x, y, z )

def genLP(Rcore, dR_Ain, dR_B, dR_Aout, sigmabead, nAin, nAout, nB):  
        # core radius, inner A layer thickness, B layer thickness, outer A layer thickness, 
        # bead diameter, # of inner A beads, # of outer A beads, # of B beads

        ntot = nAin+nB+nAout
        power = 2
        r = np.zeros((1, 3, ntot))
        types = np.zeros((ntot))

        ### Create configuration for each replicate with dispersity ###
        for step in range(0, 1):
            ### Populate A inner Layer ###
            x, y, z = gen_layer(Rcore, Rc

### converttoIQ.py

<img src="code_diagram6.png" width=600 height=600 />

**Back to getting the computed $I_{comp}$(q)**

Now that we have the scatterer positions from the `gen_LP` function, **converttoIQ.py** can call the `LPOmega` function and pass the scatterer coordinates to it to get the intra-vesicle structure factor, $\omega(q)$. Recall the set of equations to get $I_{comp}$(q):

$$ I_{comp}(q) = F_{M}^2(q)S_{MM}(q) + I_{background} $$

$F^2_{M}(q)$ is the form factor given by
$$ F^2_{M}(q) = \sum_{\alpha\in{A,B}} \sum_{\beta\in{A,B}} b_{\alpha}b_{\beta}F_{GA, \alpha}(q)F_{GA, \beta}(q)\omega(q) $$

So let us take a look at how $\omega(q)$ is calculated in **LPOmega.py** and how the spherical form factors $F_{GA, \alpha}(q)$ and $F_{GA, \beta}$ for A and B scatterers are calculated in **LPFbead.py** (note that in this example, A and B scatterers are the same and so we only need to do one spherical form factor calculation as $F_{GA, \alpha}(q)$ = $F_{GA, \beta}$).

### LPOmega.py

<img src="code_diagram9.png" width=600 height=600 />

Here we calculate the intra-vesicle structure factor as
$$ \omega(q) = \left< \frac{1}{N_{A}+N_{B}} \sum^{N_{A}+N_{B}}_{i=1} \sum^{N_{A}+N_{B}}_{j=1} \frac{\sin{qr_{ij}}}{qr_{ij}} \right> $$

where $N_{A}$ and $N_{B}$ are the number of scatterers in the two solvophilic A and one solvophobic B layers, respectively, and $r_{ij}$ is the distance between the pair of scatterers $i$ and $j$ in the vesicle assembly determined from the previous step in **genLP.py**.

**NOTE: This section is the computationally intensive calculation as it scales with the number of scatterers.**

Alternative options for directly obtaining $I_{comp}(q)$ for a given set of structural dimensions include using machine learning methods such as artificial neural networks; further disccusion on this can be found in reference [3].

To see the sourcecode for this function, run the cell block below:

In [19]:
print(getsource(cga.chemistries.vesicle.scatterer_generator.LPOmega))

def LPOmega(qrange, nAin, nAout, nB, r):                # qvalues number_of_B number_of_A scatterer_coordinates
    Ntot=nAin+nB+nAout                                  # Total number of scatterers to loop through
    omegaarrt=np.zeros((1,len(qrange)))                 # initiating array
    for step in range(0, 1):                            # loops through independent_scatterer_placemetns
        omegaarr=np.zeros((1,len(qrange)))              # initiating array
        rur=r[step,:,:]                                 # selects      
        for i in range(Ntot-1):                         # loops through index and all further indexes to prevent double counting 
            x = np.square(rur[0,i]-rur[0,(i+1):])
            y = np.square(rur[1,i]-rur[1,(i+1):])
            z = np.square(rur[2,i]-rur[2,(i+1):])
            rij = np.sqrt(np.sum([x,y,z],axis=0))       # calculates the distances
            rs = rij[:,np.newaxis]                      # reshapes array for consistency
        

### LPFbead.py

<img src="code_diagram8.png" width=600 height=600 />

The **spherical form factor amplitude** is calculated as

$$ F_{GA, \alpha}(q) = 3 \frac{ \sin(q0.5\sigma_{GA, \alpha}) - q0.5\sigma_{GA, \alpha}\cos(q0.5\sigma_{GA, \alpha}) }{(q0.5\sigma_{GA, \alpha})^3} $$

where $\alpha$ indicates A or B scatterer and $\sigma_{GA, \alpha}$ is the scatterer diameter.

To see the sourcecode for this function, run the following cell block:

In [20]:
print(getsource(cga.chemistries.vesicle.scatterer_generator.LPFbead))

def LPFbead(qrange, sigmabead):
    '''
    Compute the spherical form factor given a range of q values.
    
    Parameters:
    ----------
    qrange: numpy.array
        array of values in q-space to compute form factor for.
    sigmabead: float
        diameter of the sphere.
    
    Return:
    ----------
    Fqb: numpy.array
        array of values of the spherical form factors (F(q)) computed at q-points listed in qrange.
    '''
    
    R=np.true_divide(sigmabead,2)
    QR=np.multiply(qrange,R)
    Fqb=np.multiply(np.true_divide(np.sin(QR)-np.multiply(QR,np.cos(QR)),np.power(QR,3)),3)  

    return Fqb



### converttoIQ.py

<img src="code_diagram6.png" width=600 height=600 />

**Back again to getting the computed scattering intensity $I_{comp}$(q)**

Now that we have both $\omega(q)$ and $F_{GA, \alpha}(q)$ from the `LPOmega` and `LPFbead` functions, all that is left to do is to put together all of the pieces together to get $I_{comp}$(q) as:

$$ I_{comp}(q) = F_{M}^2(q)S_{MM}(q) + I_{background} $$

$F^2_{M}(q)$ is the form factor given by
$$ F^2_{M}(q) = \sum_{\alpha\in{A,B}} \sum_{\beta\in{A,B}} b_{\alpha}b_{\beta}F_{GA, \alpha}(q)F_{GA, \beta}(q)\omega(q) $$

and **converttoIQ.py** will output the computed scattering intensity $I_{comp}$(q) as `IQid` in the code.

Run the cell block below to see the sourcecode for this function:

In [21]:
print(getsource(cga.chemistries.vesicle.scatterer_generator.scatterer_generator.converttoIQ))

    def converttoIQ(self, qrange, param):
        '''
        Calculate computed scattering intensity profile.

        Parameters
        ----------
        qrange: int
            q values.
        param: int
            Decoded parameters.

        Return
        ------
        IQid: A numpy array holding I(q).
        '''
        # q values, decoded parameters, 
        # number of repeat units per chain, fraction of B beads per chain, core density, 
        # scatterer diameter, molar mass of B chemistry, 
        # length of A chemistry bond, length of B chemistry bond, 
        # number of scatterers per chain, # of replicates, stdev in Rcore size
        sigmabead = self.sigmabead
        N = self.N
        fb = self.fb
        rho_B = self.rho_B
        MB = self.MB
        lmono_a = self.lmono_a
        lmono_b = self.lmono_b
        num_scatterers = self.num_scatterers
        nLP = self.nLP
        
        IQid=np.zeros((len(qrange)))      #initiates array for output IQ

 

### fitness.py

<img src="code_diagram4.png" width=600 height=600 />

Having gone through getting the computed scattering intensity, we are finally back to the `fitness` function, which if the user recalls is given by the following set of equations

$$ fitness = X(sse_{max}-see)+Y $$

$$ sse = \sum w\left[log\left(\frac{I_{exp}(q)}{I_{comp}(q)}\right)\right]^2 $$

$$ X = (cs-1)\frac{max(sse_{max}-sse)}{max(sse_{max}-sse)-average(sse_{max}-sse)} $$

$$ Y = (1-X)average(sse_{max}-sse) $$

where we get the $I_{exp}$(q) from reading in the input scattering data file and $I_{comp}$(q) from the `converttoIQ` function, and in this example we choose the scaling constant $cs$ to be 10.

Within the `fitness` function, we also calculate the selection probability for each individual, $P_{select}$, that is proportional to its fitness value.
We then store the selection information (`pacc`) for all of the individuals in the population and pass it back to the main GA code where individuals will be selected to move on to the next generation.

To see the sourcecode of this function, run the following cell block:

In [22]:
print(getsource(cga.Model.fitness))

    def fitness(self,pop,generation,output_dir,metric='log_sse'):
        tic = time.time()
        cs=10
        F1= open(output_dir+'z_temp_results_'+str(generation)+'.txt','w')
        np.savetxt(output_dir+'z_temp_population_'+str(generation)+'.txt',np.c_[pop])
        
        fitn=np.zeros(self.popnumber)
        fitnfr=np.zeros(self.popnumber)
        fit=np.zeros(self.popnumber)
        qfin=self.qrange[-1]
        IQid_str=[]
        params=[]
        for val in range(self.popnumber):
            sys.stdout.write("\rGen {:d}/{:d}, individual {:d}/{:d}".format(generation+1,self.generations,val+1,self.popnumber))
            sys.stdout.flush()
            param=utils.decode(pop, val, self.nloci, self.minvalu, self.maxvalu) # gets the current structure variables
            params.append(param)

            ### calculate computed Icomp(q) ###
            IQid=self.scatterer_generator.converttoIQ(self.qrange, param)

            err=0
            for qi,qval in enumerate(self.qran

### GA_main.py

<img src="code_diagram2.png" width=600 height=600 />

Now we can finally run the main GA code (having defined the `fitness` function and all other functions that it calls) for the initial population of individuals (0-th generation).

Calling the `fitness` function we will have the fitness value for each of the individuals. In this example, the `fitness` function also provides useful information like which one of the individuals performed the best ("elite"), in terms of having its computed $I_{comp}$(q) matching the input $I_{exp}$(q), the second best ("second"), averagely ("average") and the worst ("minfit") that are stored and saved for the user.

**Selection Process: Crossover, Mutation and Elitism**

Now that we have the fitness value for each individual in the population, the GA will need to select the candidates for the next generation, and this iterative process is continued until the maximum number of generations (set by the user during the beginning of the main GA code) is reached. As each of the GA parameters is encoded as a seven-digit binary sequence, the genetic operations (crossover, mutation) performed on selected individuals is done in binary form.

*Crossover*

During crossover, two individuals in the population are selected as "parents" and their traits are mixed to produced a new "offspring" individual to be included in the next generation. Each parent individual is selected with probability $PC$ as and the offspring has traits randomly chosen from its two parent individuals. 

*Mutation*

An individual is selected for mutation with probability $PM$, where one of its traits is randomly selected to be changed within that parameter's range of values.

*Elitism*

The best individual in the generation is always included in the population for next generation.

The probabilities of crossover $PC$ and mutation $PM$ are adjusted during the iterative process over the generations during a GA run to help prevent premature convergence to local minima. When $GDM$ drops below the lower bound, then $PC$ is increased by the factor $kGDM$ (here chosen to be 1.1) and $PM$ is decreased by the same factor $kGDM$ to reduced the genetic diversity. If $GDM$ is close to 1, then $PC$ is decreased by the factor $kGDM$ and $PM$ is increased by the factor $kGDM$ to increase the genetic diversity.

**Running the GA**

After selecting the individuals in the population that get to move on to the next generation, we once again calculate the fitness for each individual in the new generation. Then the selection and fitness processes are repeated until the end of the GA. 

Run the cell block below to see the source code for the selection process:

In [23]:
print(getsource(cga.Model.genetic_operations))

    def genetic_operations(self,pop,pacc,elitei):
        popn = np.zeros(np.shape(pop))
        cross = 0
        mute = 0
        pc = self.adaptation_params.pc
        pm = self.adaptation_params.pm
        
        for i in range(self.popnumber-1):

            #####################    Crossover    ####################

            #Selection based on fitness
            testoff=random.random()
            isit=0
            npart1=1
            for j in range(1,self.popnumber):
                if (testoff>pacc[j-1])&(testoff<pacc[j]):
                    npart1=j

            testoff=random.random()
            isit=0
            npart2=1
            for j in range(self.popnumber):
                if (testoff>=pacc[j-1])&(testoff!=pacc[j]):
                    npart2=j

            #Fit parents put in array popn

            popn[i,:]=pop[npart1,:]


            testoff=random.random()
            loc=int((testoff*(self.numvars-1))*self.nloci)
            if loc==0:
              

If all of the cell blocks of code above have been run properly, then performing the GA on the small number of individuals and generations will produce the example dimensions and $I_{comp}$(q) that the user can see can compare to the target dimensions and input $I_{exp}$(q) as shown below

Good luck!

<img src="short_run_output.png" width=300 height=300 />

In [ ]:
m.solve()

### GA for Vesicles with Larger Population Size and Number of Generations

If the above codes for the example vesicle system in this tutorial have been successfully run, then the user should be able to see the best individual's $I_{comp}$(q) plotted against the input $I_{exp}$(q). With the small population size (`popnumber = 5`) and number of generations (`generations = 5`) used in this tutorial example, the resulting best individual structure is unlikely to be a good match to the target $I_{exp}$(q). As mentioned previously, using a small number of individuals and generations in this example is to keep the calculations computationally feasible and to produce accurate matches to the target $I_{exp}$(q) one would need to use significantly higher number of individuals and generations on more powerful computer.

Shown below is an example of the kind of GA determined results that we can expect when we use the GA on the same target $I_{exp}$(q) but with a much larger population size and number of generations:

`popnumber = 80`

`generations = 100`

<img src="crease_tutorial_fig_Icomp-vs-gen.png" width=300 height=300 />

**Figure 1**: *CREASE analysis on an example vesicle with the target dimensions of small core with thin B layer (shown schematically). $I_{comp}$(q) of the best individual (colored lines) from all generations in one example GA run (with dispersity) compared with the input in silico scattering intensity $I_{exp}$(q) (open circles). Progression from the initial generation to the final generation is shown with increasingly warmer colors.*

From the best individual's $I_{comp}$(q) over multiple generations, we can see that increasing the population size and number of generation gradually drives the GA determined vesicle structure towards that of the target structure (**Figure 1**). We can also make a direct comparison of the GA determined vesicle dimensions to the known target dimensions and also to the dimensions determined through the traditional analytical model fitting of scattering profiles using packages like SASVIEW.

<img src="crease_tutorial_fig_dim-ga-vs-sasview.png" width=300 height=300 />

**Figure 2**: *GA determined dimensions of relevant vesicle features at the end of 100 generations for the cases where dispersity in dimensions is not accounted for in the GA step (blue triangles) and when dispersity is incorporated into the GA (orange squares). The target dimensions of the in silico Iexp(q) are shown as open circles with 20% dispersity in the core radius (Rcore) dimensions. SASVIEW fitting results using the core-multi-shell model (grey stars) are offset to the right of the target dimensions.*

As discussed during the tutorial, GA for vesicles can incorporate the effect of dispersity in the dimensions of the assembly into the calculation of the individual's $I_{comp}$(q) and this dispersity is determined as one of the GA results given as $\sigma_{R}$ in this example systems where dispersity is incorporated in the core radius dimension. **Figure 2** shows how the GA with (i.e. GA searches and optimizes for $\sigma_{R}$) and without (i.e. GA sets $\sigma_{R} = 0$) dispersity performs in getting the vesicle structural dimensions of core radius ($R_{core}$), each layer thickness ($t_{A,in}$, $t_{B}$, $t_{A,out}$) and total vesicle diameter size ($D_{vesicle}$). We can see that with an appropriate population size and number of generations, the GA is able to accurately capture the target structure's dimensions. Additionally, while the analytical fitting to a core-multi-shell model using SASVIEW can also determine these structural dimensions, GA for vesicle is able to inform the user how the molecules are split between the inner and outer layers of the vesicle wall ($s_{Ain}$), a piece of molecular information not available to traditional method of analytical fitting. The GA determined parameter $s_{Ain}$ gives the fraction of molecules in the vesicle assembly that occupies the inner layer of the vesicle walls. For this example system, the target structure has 20% of the molecules in the assembly occupying the inner layer of the vesicle wall and the GA is able to determine $s_{Ain}$ as 28%, well matching the target value.

For more detailed discussions on CREASE for vesicles and its application to different vesicle sizes and high dispersity, the user can refer to reference [5].